In [1]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from deep_translator import GoogleTranslator

In [2]:
# Instalasi NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dfadh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dfadh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [3]:
# Fungsi untuk membersihkan teks
def preprocess_text(text):
    # Membersihkan karakter aneh dengan ekspresi reguler
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Menghilangkan tautan (URL)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Menghilangkan username atau nama akun dari teks
    text = re.sub(r'@\w+', '', text)

    # Mengubah teks menjadi huruf kecil
    text = text.lower()

    # Menghilangkan "rt"
    text = re.sub(r'\brt\b', '', text)

    # Menghilangkan karakter khusus dan angka menggunakan regex
    text = re.sub(r'[^\w\s]', '', text)

    # Menghilangkan hashtag (#)
    text = re.sub(r'#\w+', '', text)

    # Tokenisasi kata
    words = word_tokenize(text)

    # Menghilangkan stopwords (kata-kata yang tidak berarti)
    stop_words = set(stopwords.words('indonesian'))
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]

    return ' '.join(words)

In [4]:
# Fungsi untuk mendapatkan sentimen menggunakan VADER Sentiment Analyzer
def get_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    if polarity_scores['compound'] >= 0.05:
        return 'positif'
    elif polarity_scores['compound'] <= -0.05:
        return 'negatif'
    else:
        return 'netral'

In [6]:
# Membaca data dari file CSV
df = pd.read_csv('C:/Users/dfadh/Downloads/Analisis sentimen/file_potongan_1 (1).csv', encoding='utf-8')

In [7]:
df.head()

,Pilpres 2024,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,10000,rt,RT @bachrum_achmadi: Said Iqbal Partai Buruh D...,2023-04-28 07:26:12,Twitter,Positive,@donnyx17 (donnyx abu hayder),15,122,0
1,10001,mention,"Banyak sliweran capres cawapres ,,hemm",2023-04-28 07:26:12,Twitter,Positive,@oxtutiexx235 (mboketata),459,0,0
2,10002,rt,RT @tvOneNews: Kemarin PSI Bangga Deklarasikan...,2023-04-28 07:26:11,Twitter,Positive,@DadangBudiman14 (Dadang Budiman),26,122,0
3,10003,mention,Aq pilih Ã¢ÂÂ¦@ganjarpranowoÃ¢ÂÂ© lah krn di...,2023-04-28 07:26:08,Twitter,Positive,@ichwankalimasad (IchwanKalimasada),6556,0,0
4,10004,reply,@Fahrihamzah Semoga Capres dan Cawapres kali i...,2023-04-28 07:26:07,Twitter,Positive,@Andi_futago (@lif),22,0,0


In [8]:
# Pra-pemrosesan teks
df['Mentions'] = df['Unnamed: 2'].apply(preprocess_text)


In [9]:
# Terjemahkan teks dari bahasa Indonesia ke bahasa Inggris
df['Translated_Text'] = df['Mentions'].apply(lambda x: GoogleTranslator(source='id', target='en').translate(x))

In [10]:
# Menentukan sentimen menggunakan VADER Sentiment Analyzer pada teks terjemahan
df['Sentiment'] = df['Translated_Text'].apply(get_sentiment)

In [11]:
# Menampilkan hasil
print(df[['Mentions', 'Sentiment']])

                                               Mentions Sentiment
0     said iqbal partai buruh dukung ganjar capres pdip   positif
1                  banyak sliweran capres cawapres hemm    netral
2     kemarin psi bangga deklarasikan ganjar capres ...   positif
3     aq pilih lah krn dialah yg terbaik dari ketiga...   positif
4     semoga capres dan cawapres kali ini betul puny...   positif
...                                                 ...       ...
9995  4 perbuatan tak netral jokowi di pilpres 2024 ...   positif
9996  kebencian si palsu ijazah berlanjut dengan kal...   negatif
9997  jadi gini anies itu katanya elektabilitas hasi...   negatif
9998  linda menyampaikan sejumlah pertimbangan pihak...   positif
9999  orang pdip sebut sby harus netral di pilpres k...    netral

[10000 rows x 2 columns]


In [11]:
# Simpan DataFrame ke dalam file CSV
df.to_csv('hasil__Analisis.csv', index=False)